In [41]:
import pandas as pd
dataset = pd.read_csv("Preprocessed_RestaruantData.csv")
dataset

,Unnamed: 0,Rating,Seating Capacity,Average Meal Price,Marketing Budget,Social Media Followers,Chef Experience Years,Number of Reviews,Avg Review Length,Ambience Score,...,Weekday Reservations,Revenue,Location_Rural,Location_Suburban,Cuisine_French,Cuisine_Indian,Cuisine_Italian,Cuisine_Japanese,Cuisine_Mexican,Parking Availability_Yes
0,0,4.0,38,73.98,2224.0,23406.0,13,185,161.924906,1.3,...,4,6.389455e+05,1,0,0,0,0,1,0,1
1,1,3.2,76,28.11,4416.0,42741.0,8,533,148.759717,2.6,...,6,4.902078e+05,0,0,0,0,0,0,1,1
2,2,4.7,48,48.29,2796.0,37285.0,18,853,56.849189,5.3,...,14,5.413686e+05,1,0,0,0,1,0,0,0
3,3,4.4,34,51.55,1167.0,15214.0,13,82,205.433265,4.6,...,17,4.045568e+05,1,0,0,0,1,0,0,1
4,4,4.9,88,75.98,3639.0,40171.0,9,78,241.681584,8.6,...,26,1.350758e+06,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8363,8363,3.4,54,34.85,1102.0,11298.0,11,380,253.919515,9.5,...,0,4.346535e+05,0,1,0,1,0,0,0,1
8364,8364,3.7,49,36.88,1988.0,20432.0,9,713,175.590195,2.7,...,21,4.149779e+05,1,0,0,1,0,0,0,0
8365,8365,4.7,88,46.87,5949.0,63945.0,6,436,222.953647,4.8,...,21,9.303959e+05,0,0,0,0,1,0,0,1
8366,8366,3.1,31,44.53,707.0,7170.0,1,729,178.482851,6.1,...,21,3.114935e+05,1,0,0,0,0,0,0,0


In [42]:
dataset.columns

Index(['Unnamed: 0', 'Rating', 'Seating Capacity', 'Average Meal Price',
       'Marketing Budget', 'Social Media Followers', 'Chef Experience Years',
       'Number of Reviews', 'Avg Review Length', 'Ambience Score',
       'Service Quality Score', 'Weekend Reservations', 'Weekday Reservations',
       'Revenue', 'Location_Rural', 'Location_Suburban', 'Cuisine_French',
       'Cuisine_Indian', 'Cuisine_Italian', 'Cuisine_Japanese',
       'Cuisine_Mexican', 'Parking Availability_Yes'],
      dtype='object')

# Spilt input and output

In [43]:
indep = dataset[['Rating', 'Seating Capacity', 'Average Meal Price',
       'Marketing Budget', 'Social Media Followers', 'Chef Experience Years',
       'Number of Reviews', 'Avg Review Length', 'Ambience Score',
       'Service Quality Score', 'Weekend Reservations', 'Weekday Reservations', 'Location_Rural', 'Location_Suburban', 'Cuisine_French',
       'Cuisine_Indian', 'Cuisine_Italian', 'Cuisine_Japanese',
       'Cuisine_Mexican', 'Parking Availability_Yes']]
dep = dataset[['Revenue']]

# Feature Selection

In [44]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
test = SelectKBest(score_func=f_regression, k=4)
fit1= test.fit(indep,dep)
# summarize scores       
selectk_features = fit1.transform(indep)
mask = fit1.get_support() #get_support() returns a boolean array where each element is True if the corresponding feature was selected and False otherwise.
selected_features = indep.columns[mask] #The boolean mask is used to filter these column names, so only the names of the selected features are retaine

C:\Users\nravi378\AppData\Local\anaconda3\envs\art\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Standardisation for input and output

In [45]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
indep_X= sc.fit_transform(selectk_features)

scy = StandardScaler()
dep_Y = scy.fit_transform(dep)

# Model Creation

In [46]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
param_grid = {'kernel':['rbf','sigmoid'],'C':[0.1,10,100,1000]}
grid = GridSearchCV(SVR(),param_grid,refit = True,verbose=3,n_jobs=-1)
SVR_regressor=grid.fit(indep_X,dep_Y)
SVR = SVR_regressor.best_score_
print(SVR)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
0.9985354093286357


C:\Users\nravi378\AppData\Local\anaconda3\envs\art\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [47]:
from sklearn.linear_model import LinearRegression
param_grid = {}
grid3 = GridSearchCV(LinearRegression(),param_grid,refit = True,verbose=3,n_jobs=-1)
MLR_regressor=grid3.fit(indep_X,dep_Y)
MLR = MLR_regressor.best_score_
print(MLR)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
0.9591252720412127


In [48]:
from sklearn.ensemble import RandomForestRegressor
param_grid = { 'criterion' : [ 'squared_error' , 'absolute_error', 'friedman_mse', 'poisson'] , 'n_estimators':[20,50,100] ,'random_state' : [0] }
grid1 = GridSearchCV(RandomForestRegressor(),param_grid,refit = True,verbose=3,n_jobs=-1)
RF_regressor=grid1.fit(indep_X,dep_Y)
RF = RF_regressor.best_score_
print(RF)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\nravi378\AppData\Local\anaconda3\envs\art\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\nravi378\AppData\Local\anaconda3\envs\art\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\nravi378\AppData\Local\anaconda3\envs\art\lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\nravi378\AppData\Local\anaconda3\envs\art\lib\site-packages\sklearn\ensemb

0.9990739590368511


In [49]:
from sklearn.tree import DecisionTreeRegressor
param_grid = { 'criterion' : [ 'squared_error' , 'absolute_error', 'friedman_mse', 'poisson'] , 'splitter': ['best', 'random'] }
grid2 = GridSearchCV(DecisionTreeRegressor(),param_grid,refit = True,verbose=3,n_jobs=-1)
DT_regressor=grid2.fit(indep_X,dep_Y)
DT = DT_regressor.best_score_
print(DT)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
0.9982197943496057


C:\Users\nravi378\AppData\Local\anaconda3\envs\art\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
10 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\nravi378\AppData\Local\anaconda3\envs\art\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\nravi378\AppData\Local\anaconda3\envs\art\lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\nravi378\AppData\Local\anaconda3\envs\art\lib\site-packages\sklearn\tree\_

In [50]:
ResultTable = pd.DataFrame(columns = ["MLR","SVM","DT","RF"], index = ["f1_regression"])

ResultTable["MLR"]["f1_regression"] = MLR
ResultTable["SVM"]["f1_regression"] = SVR
ResultTable["DT"]["f1_regression"] = DT
ResultTable["RF"]["f1_regression"] = RF


ResultTable


C:\Users\nravi378\AppData\Local\Temp\ipykernel_13704\3972211358.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  ResultTable["MLR"]["f1_regression"] = MLR
C:\Users\nravi378\AppData\Local\Temp\ipykernel_13704\3972211358.py:4: FutureWarning

,MLR,SVM,DT,RF
f1_regression,0.959125,0.998535,0.99822,0.999074


# Choosing Random Forest for having nearly R2 score value is 1.

In [51]:
import pickle
filename = "BestModel_for_RestaurantRevenue.sav"
pickle.dump(RF_regressor,open(filename,"wb"))

In [52]:
from sklearn.preprocessing import StandardScaler
pre_input=sc.transform([[34,59,0,1]])

In [56]:
Loaded_model = pickle.load(open("BestModel_for_RestaurantRevenue.sav","rb"))
output_std = Loaded_model.predict(pre_input)
Result = scy.inverse_transform([output_std])
Result

array([[468944.8653]])

In [57]:
selected_features

Index(['Seating Capacity', 'Average Meal Price', 'Location_Rural',
       'Cuisine_Japanese'],
      dtype='object')